In [ ]:
!pip install textblob

In [ ]:
!pip install tweepy

In [ ]:
!pip install pycountry

In [ ]:
!pip install wordcloud 

In [ ]:
!pip install langdetect

In [34]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
import config
# put ur keys in a py file as a dictionary and import them so u can keep em secret
consumer_key = config.params['consumer_key'] #api key 
consumer_secret = config.params['consumer_secret'] #api secret 
access_token = config.params['access_token'] #access token  
access_token_secret = config.params['access_token_secret'] #access secret  
bear = config.params['bear'] #bearer token

In [36]:
client = tweepy.Client(bearer_token=bear,\
                       consumer_key = consumer_key,\
                       consumer_secret = consumer_secret,\
                       access_token = access_token,\
                       access_token_secret = access_token_secret)
query = 'NFL'
tweets = client.search_recent_tweets(query=query, max_results=100)
tweet_list = []
for tweet in tweets.data:
    tweet_list.append(tweet.text)
tweet_list = pd.DataFrame(tweet_list)
tweet_list


,0
0,@chopchop2115 @ProFootballDoc @NFL Asking the ...
1,@BaldyNFL @dallascowboys Thanks for your insig...
2,RT @RedditCFB: Mack Brown’s final year feature...
3,RT @espn: Which team that selected a QB on Day...
4,Perfis em destaque:\n#bourguignon_08\n#piadas_...
...,...
95,RT @thepiff: Just bringing the ✨magic✨ to the ...
96,RT @TheMozKnowz: .@DrewJRosenhaus tells me he ...
97,"@HuskiesRock2020 @WestCoastCFB So, you're clai..."
98,"RT @johnpboyle: The #Seahawks ""attacked some n..."
